**Упражнение 1.** Запустите код. Попробуйте объяснить, почему LIST - пуст.

In [10]:
import multiprocessing

def worker():
    LIST.append('item')

LIST = []

if __name__ == "__main__":
    processes = [
        multiprocessing.Process(target=worker)
        for _ in range(5)
    ]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    print(LIST)

[]


**Объяснение:**

В отличие от потоков, процессы обладают собственной областью памяти и не имеют доступа к памяти родительского процесса. По этой причине код, запущенный с помощью `multiprocessing.Process` не сможет мутировать список `LIST`. Ниже приведен код, реализованный на потоках, который приводит к ожидаемому результату (массив `LIST` наполняется 5 элементами).

In [23]:
import threading

def worker():
    LIST.append('item')

LIST = []

if __name__ == '__main__':
    threads = [threading.Thread(target=worker) for _ in range(5)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    print(LIST)

['item', 'item', 'item', 'item', 'item']


**Упражнение 2.** Запустите код. Какая проблема здесь возникает? Исправьте её.

In [20]:
import threading
import time

l1 = threading.Lock()
l2 = threading.Lock()

def f1(name):
    print('thread',name,'about to lock l1')
    with l1:
        print('thread',name,'has lock l1')
        time.sleep(0.3)
        print('thread',name,'about to lock l2')
        with l2:
            print('thread',name)

def f2(name):
    print('thread',name,'about to lock l2')
    with l1:
        print('thread',name,'has lock l2')
        time.sleep(0.3)
        print('thread',name,'about to lock l1')
        with l2:
            print('thread',name)

if __name__ == '__main__':
    t1=threading.Thread(target=f1, args=['t1',])
    t2=threading.Thread(target=f2, args=['t2',])

    t1.start()
    t2.start()

    t1.join()
    t2.join()

thread t1 about to lock l1
thread t1 has lock l1
thread t2 about to lock l2
thread t1 about to lock l2
thread t1
thread t2 has lock l2
thread t2 about to lock l1
thread t2


**Объяснение:**

Код, приведенный в упражнении, работает "как надо", т.е. оба процесса успешно выполняют все свои операции и в программе не возникает взаимных блокировок. Тем не менее, в целях выполнения данной лабораторной работы, можно представить альтернативную ситуацию, в которой порядок захвата мьютексов отличается между потоками. Код, описывающий такую ситуацию, приведен ниже. Обратите внимание, что этот выполнение этого кода никогда не закончится из-за возникающей в нем взаимной блокировки.

In [21]:
from threading import Lock, Thread
from time import sleep

l1, l2 = Lock(), Lock()


def f1(name):
    print('thread', name, 'about to lock l1')
    with l1:
        print('thread', name, 'has lock l1')
        sleep(0.3)
        print('thread', name, 'about to lock l2')
        with l2:
            print('thread', name)


def f2(name):
    print('thread', name, 'about to lock l2')
    with l2:
        print('thread', name, 'has lock l2')
        sleep(0.3)
        print('thread', name, 'about to lock l1')
        with l1:
            print('thread', name)


if __name__ == '__main__':
    threads = [Thread(target=f1, args=['t1',]),
               Thread(target=f2, args=['t2',])]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


thread t1 about to lock l1
thread t1 has lock l1
thread t2 about to lock l2
thread t2 has lock l2
thread t1 about to lock l2
thread t2 about to lock l1


KeyboardInterrupt: 

Одним из способов избежать появления взаимной блокировки в данном случае является добавление третьего объекта `Lock()`, ограничивающего доступ к ресурсам, контролируемым объектами `l1` и `l2`.

In [22]:
from threading import Lock, Thread
from time import sleep

l1, l2 = Lock(), Lock()

l3 = Lock()


def f1(name):
    with l3:
        print('thread', name, 'about to lock l1')
        with l1:
            print('thread', name, 'has lock l1')
            sleep(0.3)
            print('thread', name, 'about to lock l2')
            with l2:
                print('thread', name)


def f2(name):
    with l3:
        print('thread', name, 'about to lock l2')
        with l2:
            print('thread', name, 'has lock l2')
            sleep(0.3)
            print('thread', name, 'about to lock l1')
            with l1:
                print('thread', name)


if __name__ == '__main__':
    threads = [Thread(target=f1, args=['t1',]),
               Thread(target=f2, args=['t2',])]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


thread t1 about to lock l1
thread t1 has lock l1
thread t1 about to lock l2
thread t1
thread t2 about to lock l2
thread t2 has lock l2
thread t2 about to lock l1
thread t2


**Упражнение 3.** Вам необходимо вычислить значение функции f = x**2 + x * 2 + 10 * x для различных значений аргументов (10 случайных чисел).

1. Сделайте программу без использования потоков/процессов и измерьте время.

2. Разбейте задачу на несколько потоков (отдельный поток для каждого аргумента и отдельный поток для каждого слагаемоего функции). Запустите программу. Измерьте время. 

3. Разбейте задачу на несколько процессов (отдельный процесс для каждого аргумента и отдельный процесс для каждого слагаемоего функции). Запустите программу. Измерьте время. 

Для синхронизации можно использовать барьеры.

Объясните полученные результаты.

In [4]:
from random import uniform
from threading import Thread, Lock
from multiprocessing import Process, Value

def work_single_thread():
    def calculate(x):
        return x ** 2 + 2 * x + 10 * x

    args = [uniform(0, 100) for _ in range(10)]

    for x in args:
        calculate(x)


def work_multithread():
    def calculate_multithread(x):
        result = 0
        lock = Lock()

        def task_1():
            nonlocal result
            part = x ** 2
            with lock:
                result += part

        def task_2():
            nonlocal result
            part = 2 * x
            with lock:
                result += part

        def task_3():
            nonlocal result
            part = x * 10
            with lock:
                result += part

        threads = [Thread(target=task) for task in [task_1, task_2, task_3]]
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()

        return result

    args = [uniform(0, 100) for _ in range(10)]
    threads = [Thread(target=calculate_multithread, args=(x, )) for x in args]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


def work_multiprocess():
    def calculate_multiprocess(x):
        def task_1(x, r):
            part = x ** 2
            with r.get_lock():
                r.value += part

        def task_2(x, r):
            part = 2 * x
            with r.get_lock():
                r.value += part

        def task_3(x, r):
            part = x * 10
            with r.get_lock():
                r.value += part

        result = Value('d', 0.0)

        processes = [
            Process(target=task_1, args=(x, result)),
            Process(target=task_2, args=(x, result)),
            Process(target=task_3, args=(x, result)),
        ]
        for process in processes:
            process.start()
        for process in processes:
            process.join()

        return result.value

    args = [uniform(0, 100) for _ in range(10)]
    processes = [Process(target=calculate_multiprocess, args=(x, ))
                 for x in args]
    for process in processes:
        process.start()
    for process in processes:
        process.join()

In [6]:
%timeit work_single_thread()
%timeit work_multithread()
%timeit work_multiprocess()

4.97 µs ± 84.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.64 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
220 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Объяснение:**

Многопоточная реализация решения задачи не может привести к приросту в производительности, так как приведенная относится к CPU-bound-задачам.

Многопроцессная реализация задачи также не улучшает время исполнения программы, так как из-за простоты приведенной задачи издержки на использование многопроцессности превышают выгоду от ее распараллеливания.

**Упражнение 4.** Смоделируйте следующую ситуацию с использованием семафоров и событий.

1. Есть 5 касс и 20 покупателей. Все они хотят купить билеты на матч. На билетах не указаны места. 

2. После покупки билета покупатель бежит на стадион (у каждого разная скорость бега, это намек, что надо сделать sleep) и занимает свободное место.

Программа должна вывести логированные события по типу:

client 0, service time (ticket): 1.0004174709320068

client 1, service time (ticket): 1.0005174709320068

client 1, runnig time to stad:   1.2004174709320068

client 0, runnig time to stad:   1.5004174709320068

In [19]:
from threading import Thread, Semaphore
from random import uniform
from time import perf_counter, sleep

cashiers_num = 5
clients_num = 20

ticket_office = Semaphore(cashiers_num)


def work(num):
    before_service = perf_counter()
    with ticket_office:
        sleep(uniform(1, 6))
    after_service = perf_counter()
    print(f'client {num}\twait time:\t{after_service - before_service}')
    sleep(uniform(1, 6))
    print(f'client {num}\tarrival time:\t{perf_counter() - after_service}')


threads = [Thread(target=work, args=(i,)) for i in range(clients_num)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

client 2	wait time:	1.1386303360341117
client 0	wait time:	1.356558125000447
client 4	wait time:	2.3463734409888275
client 1	wait time:	2.5756050089839846
client 0	arrival time:	2.7315891530015506
client 7	wait time:	4.165329527982976
client 3	wait time:	4.567125396977644
client 6	wait time:	4.567115792015102
client 4	arrival time:	2.300787203013897
client 7	arrival time:	1.2854235660051927
client 2	arrival time:	4.688847080979031
client 10	wait time:	6.114432237984147
client 5	wait time:	6.199242699018214
client 3	arrival time:	1.7743961600353941
client 8	wait time:	6.77484273299342
client 6	arrival time:	2.2397508029825985
client 1	arrival time:	4.343860118009616
client 9	wait time:	7.017885534034576
client 12	wait time:	7.449067716021091
client 11	wait time:	8.574841618014034
client 5	arrival time:	2.7197922699851915
client 10	arrival time:	3.043971061008051
client 12	arrival time:	2.012600790010765
client 9	arrival time:	3.388867475965526
client 14	wait time:	10.990331290988252
cli

**Упражнение 5.** Покажите и опишите разницу между Semaphore и BoundedSemaphore.

Объекты `threading.Semaphore` и `threading.BoundedSemaphore` используются для ограничения числа потоков, имеющих доступ к секции кода, и используют внутренний счетчик `_value`, отражающий количество "свободных мест" и инициализируемый при создании объекта. 

Счетчик семафора уменьшается при вызове метода `acquire()` и увеличивается при вызове метода `release()`. 

Вызов `acquire()` при `_value == 0` приведет к остановке выполнения кода, вызвавшего метод, пока другой поток не вызовет `release()`, тем самым увеличив значение `_value`.

Разница между `Semaphore` и `BoundedSemaphore` проявляется в том, как они обрабатывают вызов `release()` при значении `_value`, равному исходному значению (определенному при инициализации).

Вызов `Semaphore.release` при таком `_value` будет обработан как обычно и приведет к увеличению `_value` выше исходного значения.

Вызов `BoundedSemaphore.release` в такой же ситуации выбросит исключение `ValueError`, предотвратив увеличение `_value` выше исходного значения.

In [17]:
from threading import Semaphore, BoundedSemaphore

s = Semaphore(3)
b = BoundedSemaphore(3)

print(f'Initial Semaphore value: {s._value}')

s.acquire()
print(f'Semaphore value after acquire(): {s._value}')

s.release()
print(f'Semaphore value after release(): {s._value}')

s.release()
print(f'Semaphore value after release(): {s._value}\n')


print(f'Initial BoundedSemaphore value: {b._value}')

b.acquire()
print(f'BoundedSemaphore value after acquire(): {b._value}')

b.release()
print(f'BoundedSemaphore value after release(): {b._value}')

try:
    b.release()
    print(f'BoundedSemaphore value after release(): {b._value}')
except ValueError:
    print("Cannot release BoundedSemaphore!")
finally:
    print(f'Final BoundedSemaphore value: {b._value}')


Initial Semaphore value: 3
Semaphore value after acquire(): 2
Semaphore value after release(): 3
Semaphore value after release(): 4

Initial BoundedSemaphore value: 3
BoundedSemaphore value after acquire(): 2
BoundedSemaphore value after release(): 3
Cannot release BoundedSemaphore!
Final BoundedSemaphore value: 3


**Упражнение 6.** Запустите на исполнение, замерив время работы. Перепишите с помощью потоков и опять замерьте время. Затем с помощью процессов и снова измерьте время. Объясните результат.

In [ ]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)
print(time.time() - start)

4.498141050338745


In [11]:
from urllib.request import urlopen
from threading import Thread
from multiprocessing import Process

urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urlopen(url) as u:
        return u.read()


def read_urls_single_thread():
    for url in urls:
        read_url(url)


def read_urls_multithread():
    threads = [Thread(target=read_url, args=(url, )) for url in urls]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()


def read_urls_multiprocess():
    processes = [Process(target=read_url, args=(url, )) for url in urls]
    for process in processes:
        process.start()
    for process in processes:
        process.join()

In [13]:
%timeit read_urls_single_thread()
%timeit read_urls_multithread()
%timeit read_urls_multiprocess()

3.96 s ± 752 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.89 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.52 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Объяснение:**

Так как задача является IO-bound, использование многопоточности улучшило время выполнения задачи. Многопроцессная реализация решения также уменьшила время работы за счет параллельности выполнения.